In [ ]:
import bm25s
import os, gzip, json
from sentence_transformers import util
from sklearn.feature_extraction import _stop_words
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
wikipedia_filepath = "./data/simplewiki-2020-11-01.jsonl.gz"
embedding_save_path = "./dta/simple_wikipedia.pk"

if not os.path.exists(wikipedia_filepath):
    util.http_get("http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz", wikipedia_filepath)

passages_first_sentence = []
with gzip.open(wikipedia_filepath, "rt", encoding="utf8") as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        passages_first_sentence.append(data["paragraphs"][0])

In [ ]:
if os.path.exists(embedding_save_path):
    corpus_embeddings = torch.load(embedding_save_path)
else:
# We encode all passages into our vector space. This takes about 5 minutes (depends on your GPU speed)
    corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
    torch.save(corpus_embeddings, embedding_save_path)

In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio/flagged/dataset1.csv


## Load trec covid dataset
https://paperswithcode.com/dataset/trec-covid

In [27]:
corpus = load_dataset("BeIR/trec-covid", 'corpus')['corpus']
queries = load_dataset("BeIR/trec-covid", 'queries')['queries']
qrels = load_dataset("BeIR/trec-covid-qrels")


In [28]:
queries

Dataset({
    features: ['_id', 'title', 'text'],
    num_rows: 50
})

In [29]:
def combine_text(example):
    example['full_text'] = '[Title] ' + example['title'] + ' [TEXT] ' + example['text']
    return example
corpus = corpus.map(combine_text)

## Load model

In [30]:
model = SentenceTransformer("all-MiniLM-L6-v2")


In [31]:
corpus_embeddings = model.encode(corpus['full_text'], convert_to_tensor=True, show_progress_bar=True)


Batches: 100%|██████████| 5355/5355 [01:25<00:00, 62.98it/s] 


In [32]:
query_embeddings = model.encode(queries['text'], convert_to_tensor=True, show_progress_bar=True)


Batches: 100%|██████████| 2/2 [00:00<00:00, 100.12it/s]


In [33]:
query_embeddings.shape

torch.Size([50, 384])

In [67]:
hits = util.semantic_search(query_embeddings, corpus_embeddings, top_k=2000)

In [68]:
len(hits)

50

In [69]:
len(hits[0])

2000

In [70]:
type(corpus['_id'])

list

In [71]:

retrieval_results = collections.defaultdict(dict)
doc_id_map = corpus['_id']
query_id_map = queries['_id']
for qid, doc_score_list in enumerate(hits):
    qid_key = query_id_map[qid]
    result_dict = {}
    doc_ids = [doc_score['corpus_id'] for doc_score in doc_score_list]
    scores = [doc_score['score'] for doc_score in doc_score_list]
    
    doc_id_keys = list(map(lambda x: doc_id_map[x], doc_ids))
    result_dict = dict(zip(doc_id_keys, scores))
    retrieval_results[qid_key] = result_dict

In [72]:
qrels_for_eval = collections.defaultdict(dict)
for example in qrels['test']:
    qrels_for_eval[str(example['query-id'])][str(example['corpus-id'])] = example['score']



In [74]:
recall(qrels=qrels_for_eval, results=retrieval_results, k_values=[1, 3, 5, 10, 20, 30, 100, 500, 2000])

{'Recall@1': 0.00191,
 'Recall@3': 0.00488,
 'Recall@5': 0.00775,
 'Recall@10': 0.01431,
 'Recall@20': 0.02549,
 'Recall@30': 0.03588,
 'Recall@100': 0.09415,
 'Recall@500': 0.2493,
 'Recall@2000': 0.44927}

In [75]:
precision(qrels=qrels_for_eval, results=retrieval_results, k_values=[1, 3, 5, 10, 20, 30, 100, 500, 2000])

{'Precision@1': 0.74,
 'Precision@3': 0.64667,
 'Precision@5': 0.612,
 'Precision@10': 0.564,
 'Precision@20': 0.503,
 'Precision@30': 0.484,
 'Precision@100': 0.3974,
 'Precision@500': 0.22796,
 'Precision@2000': 0.10729}

In [79]:
ndcg(qrels=qrels_for_eval, results=retrieval_results, k_values=[1, 3, 5, 10])

{'NDCG@1': 0.67, 'NDCG@3': 0.60222, 'NDCG@5': 0.57071, 'NDCG@10': 0.53565}